In [1]:
import taichi as ti
import numpy as np
from typing import Tuple
import napari
import matplotlib.pyplot as plt

[Taichi] version 1.7.4, llvm 15.0.1, commit b4b956fd, win, python 3.11.5


In [2]:
from beamgen import EllipticalBeamGenerator
from defocus_microscope import DefocusMicroscope
from helpers import init_trajectories
from simulation import Simulation

In [ ]:
N = 128
wavelength = 0.65
N_PHOTONS = 1000000000
BATCH_SIZE = 50000000
N_TRACKED = 100
runner = Simulation(N=N, wavelength=wavelength, n_photons=N_PHOTONS, batch_size=BATCH_SIZE)

[Taichi] Starting on arch=cuda


### Define volume

In [4]:
types = [
    {"name": "cell", "rel_scale": 0.5, "rel_density": 10, "rel_pigment": 1E-10, "r_index": 0.5},
    {"name": "cell2", "rel_scale": 1, "rel_density": 5, "rel_pigment": 5E-11, "r_index": 0.5},
    {"name": "cell3", "rel_scale": 1, "rel_density": 2.5, "rel_pigment": 25E-12, "r_index": 0.5}
]

In [5]:
spheres_new = [
    {
        'center': ti.Vector([40.0, 75.0, 40.0]), 'radii': [30.0, 20.0], 'types': ["cell", "empty"]
    },
    {
        'center': ti.Vector([88.0, 88.0, 40.0]), 'radii': [25.0, 10.0], 'types': ["cell2", "empty"]
    },
    {
        'center': ti.Vector([40.0, 88.0, 88.0]), 'radii': [20.0, 18.0], 'types': ["cell3", "empty"]
    }
]

In [6]:
runner.init_types(types, 10000)

In [7]:
runner.initSpheres(spheres_new)

### Generate photons and initialize tracked values

In [8]:
# generate MC light packets
# Set up beam parameters
central_point = (64, 64, 64)  # Target at origin
distance = np.sqrt(2)*64+1  # Source 10 units away
theta = 0
phi = np.pi/2
mask_diameter = 1.5*np.sqrt(128**2)
sigma_a = 256
sigma_b = 256
divergence_a = -0.02
divergence_b = -0.02         # why is this sign reversed?
divergence_sigma_a = 0.005
divergence_sigma_b = 0.005

MAX_STEPS = round(distance * 10)
STEP_LENGTH = 0.5

runner.init_beam_generator(central_point, distance, theta, phi, mask_diameter, sigma_a, sigma_b, divergence_a, divergence_b, divergence_sigma_a, divergence_sigma_b)



In [9]:
faces = ["+x", "+y", "-z"]

for face in faces:
    runner.init_microscope(face, N, 1.0, 0.15, 1, 1.33, (N, N), 1.0, 550, 64.0)

Taichi Microscope with Defocus initialized:
  Observation face: +x
  Focal depth: 64.0 voxels (64.0 μm)
  Depth of field: 12.22 μm (12.22 voxels)
  NA: 0.15, Acceptance angle: 6.5°
  Min sigma (in-focus): 0.95 pixels
Taichi Microscope with Defocus initialized:
  Observation face: +y
  Focal depth: 64.0 voxels (64.0 μm)
  Depth of field: 12.22 μm (12.22 voxels)
  NA: 0.15, Acceptance angle: 6.5°
  Min sigma (in-focus): 0.95 pixels
Taichi Microscope with Defocus initialized:
  Observation face: -z
  Focal depth: 64.0 voxels (64.0 μm)
  Depth of field: 12.22 μm (12.22 voxels)
  NA: 0.15, Acceptance angle: 6.5°
  Min sigma (in-focus): 0.95 pixels


In [10]:
# Initialize the napari viewer
viewer = napari.Viewer(title="3D Test Volume", ndisplay=3)

### View volume and projections

In [11]:
"""
projections = runner.project_volume(["x", "y", "z"])
proj0_3d = projections["z"]
proj1_3d = projections["y"]
proj2_3d = projections["x"]
"""
runner.volume_to_np()
absorb_np = runner.absorption_np()
scatter_np = runner.scatter_np()

In [12]:
viewer.add_image(
    absorb_np, 
    name='3D Absorption', 
    colormap='turbo', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
)
viewer.add_image(
    scatter_np, 
    name='3D Scattering', 
    colormap='turbo', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
)

# add the same volume and render as plane
# plane should be in 'additive' blending mode or depth looks all wrong
render_sums = False

if render_sums:
    sum0_layer = viewer.add_image(
        proj0_3d,
        rendering='average',
        name='sum0',
        depiction='plane',
        blending='additive',
        opacity=0.25,
        plane={'position': (0, N/2, N/2),'normal': (1, 0, 0)}
    )

    sum1_layer = viewer.add_image(
        proj1_3d,
        rendering='average',
        name='sum1',
        depiction='plane',
        blending='additive',
        opacity=0.25,
        plane={'position': (N/2, 0, N/2),'normal': (0, 1, 0)}
    )

    sum2_layer = viewer.add_image(
        proj2_3d,
        rendering='average',
        name='sum2',
        depiction='plane',
        blending='additive',
        opacity=0.25,
        plane={'position': (N/2, N/2, 0),'normal': (0, 0, 1)}
    )

In [13]:
axes = np.array((
    [[0, 0, 0], [128,0,0]],
    [[0, 0, 0], [0, 128, 0]],
    [[0, 0, 0], [0, 0, 128]])
)
viewer.add_shapes(axes[0], shape_type='line', edge_width=0.2, edge_color='red', opacity=0.75, name='axis-0')
viewer.add_shapes(axes[1], shape_type='line', edge_width=0.2, edge_color='green', opacity=0.75, name='axis-1')
viewer.add_shapes(axes[2], shape_type='line', edge_width=0.2, edge_color='blue', opacity=0.75, name='axis-2')


<Shapes layer 'axis-2' at 0x21a1a69a250>

### Field Gradient Calculation

In [14]:
"""

# calculate local gradients
gradient_mag = ti.field(dtype=ti.f32, shape=volume.shape)
gradient_dir = ti.Vector.field(3, dtype=ti.f32, shape=volume.shape)

@ti.kernel
def compute_gradient():
    h = ti.static(1)
    # Precompute normalization factor sigma = sum(ox^2) over ox * (2h+1)^2
    sigma = 0.0
    for ox in ti.static(range(-h, h + 1)):
        sigma += ox * ox
    sigma *= (2 * h + 1) ** 2
    #if sigma == 0.0:  # Avoid division by zero (though N>=3)
    #    return

    for i, j, k in volume:
        # Skip boundaries (you can add padding or mirroring if needed)
        if i < h or i >= nx - h or j < h or j >= ny - h or k < h or k >= nz - h:
            gradient_mag[i, j, k] = 0.0
            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])
            continue

        gx = 0.0
        gy = 0.0
        gz = 0.0
        # Loop over neighborhood
        for ox in ti.static(range(-h, h + 1)):
            for oy in ti.static(range(-h, h + 1)):
                for oz in ti.static(range(-h, h + 1)):
                    f = volume[i + ox, j + oy, k + oz]
                    gx += ox * f
                    gy += oy * f
                    gz += oz * f

        gx /= sigma
        gy /= sigma
        gz /= sigma

        mag = ti.sqrt(gx * gx + gy * gy + gz * gz)
        gradient_mag[i, j, k] = mag

        if mag > 1e-6:
            gradient_dir[i, j, k] = ti.Vector([gx / mag, gy / mag, gz / mag])
        else:
            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])
            gradient_mag[i, j, k] = 0.0

# Usage example
nx, ny, nz = 128, 128, 128
compute_gradient()  # Change to 5, 7, etc.
if False:
    mag_np = gradient_mag.to_numpy()
    dir_np = gradient_dir.to_numpy()
    viewer.add_image(
    mag_np, 
    name='3D Index', 
    colormap='red', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
    )
    viewer.add_vectors(
        dir_np, 
        name='Gradient Dir', 
        edge_color='red'
    )
"""

"\n\n# calculate local gradients\ngradient_mag = ti.field(dtype=ti.f32, shape=volume.shape)\ngradient_dir = ti.Vector.field(3, dtype=ti.f32, shape=volume.shape)\n\n@ti.kernel\ndef compute_gradient():\n    h = ti.static(1)\n    # Precompute normalization factor sigma = sum(ox^2) over ox * (2h+1)^2\n    sigma = 0.0\n    for ox in ti.static(range(-h, h + 1)):\n        sigma += ox * ox\n    sigma *= (2 * h + 1) ** 2\n    #if sigma == 0.0:  # Avoid division by zero (though N>=3)\n    #    return\n\n    for i, j, k in volume:\n        # Skip boundaries (you can add padding or mirroring if needed)\n        if i < h or i >= nx - h or j < h or j >= ny - h or k < h or k >= nz - h:\n            gradient_mag[i, j, k] = 0.0\n            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])\n            continue\n\n        gx = 0.0\n        gy = 0.0\n        gz = 0.0\n        # Loop over neighborhood\n        for ox in ti.static(range(-h, h + 1)):\n            for oy in ti.static(range(-h, h + 1)):\n  

### Photon Interaction

In [15]:
TRACK = True
runner.simulation_loop(MAX_STEPS, STEP_LENGTH, TRACK, N_TRACKED)

Loop 1 (1000000000 photons remaining):
Running 100000000 photons

Adding photons to microscope 0 (+x):
Face +x: 1886952 photons at face, 6147 within NA
  Unscattered photons: 0 (0.0%)
  Sigma range: [0.95, 2.54] pixels
  Accumulated peak intensity: 4.45e-02

Adding photons to microscope 1 (+y):
Face +y: 2282752 photons at face, 7775 within NA
  Unscattered photons: 0 (0.0%)
  Sigma range: [0.95, 2.54] pixels
  Accumulated peak intensity: 4.17e-02

Adding photons to microscope 2 (-z):
Face -z: 43704740 photons at face, 43295486 within NA
  Unscattered photons: 43060606 (99.5%)
  Sigma range: [0.95, 2.72] pixels
  Accumulated peak intensity: 3.71e+01
Loop 2 (900000000 photons remaining):
Running 100000000 photons

Adding photons to microscope 0 (+x):
Face +x: 1889196 photons at face, 6040 within NA
  Unscattered photons: 0 (0.0%)
  Sigma range: [0.95, 2.54] pixels
  Accumulated peak intensity: 8.35e-02

Adding photons to microscope 1 (+y):
Face +y: 2285009 photons at face, 7997 within NA

RuntimeError: [taichi/rhi/cuda/cuda_driver.h:taichi::lang::CUDADriverFunction<void * *,unsigned __int64,void *>::operator ()@92] CUDA Error CUDA_ERROR_OUT_OF_MEMORY: out of memory while calling malloc_async_impl (cuMemAllocAsync)

In [ ]:
"""
runner.init_photons(N_PHOTONS)
runner.init_tracking(N_TRACKED, MAX_STEPS)
runner.run_simulation(MAX_STEPS, 0.5)
"""

'\nrunner.init_photons(N_PHOTONS)\nrunner.init_tracking(N_TRACKED, MAX_STEPS)\nrunner.run_simulation(MAX_STEPS, 0.5)\n'

In [ ]:
render_interactions = True
render_outputs = False

if render_interactions:
    interactions_np = runner.get_interactions()
    viewer.add_image(
        interactions_np, 
        name='3D Interactions', 
        colormap='turbo', 
        contrast_limits=[0, 1],
        rendering='average',
        blending='additive'
    )

if render_outputs:
    output_np = runner.get_exits()
    viewer.add_image(
        output_np, 
        name = 'Exiting Light',
        colormap='turbo',
        contrast_limits=[0, 1],
        rendering='average',
        blending='additive'
    )

### Create defocused microscope and save image projection

In [ ]:
for i, face in enumerate(faces):
    image_defocus = runner.defocus_image(i)

    if image_defocus.max() > 0:
        # Map observation face to plane position and normal
        # NOTE: Positions should be within [0, N-1] bounds
        face_config = {
            '+z': {'position': (N/2, N/2, 0), 'normal': (0, 0, -1)},   # Changed N to N-1
            '-z': {'position': (N/2, N/2, 0), 'normal': (0, 0, 1)},
            '+x': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},   # Changed N to N-1
            '-x': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},
            '+y': {'position': (N/2, 0, N/2), 'normal': (0, -1, 0)},   # Changed N to N-1
            '-y': {'position': (N/2, 0, N/2), 'normal': (0, 1, 0)},
        }

        config = face_config[face]
        
        # Reshape image to 3D for plane rendering (same as projection planes)
        if face in ['+x', '-x']:
            # z-face: image is in xy plane
            image_3d = image_defocus[np.newaxis, :, :]  # Shape: (1, Y, X)
        elif face in ['+z', '-z']:
            # x-face: image is in yz plane
            image_3d = image_defocus[:, :, np.newaxis]  # Shape: (Y, 1, Z)
        elif face in ['+y', '-y']:
            # y-face: image is in xz plane
            image_3d = image_defocus[:, np.newaxis, :]  # Shape: (X, Z, 1)
        
        viewer.add_image(
            image_3d,
            name=f'Microscope Defocus ({face})',
            colormap='gray',
            contrast_limits=[0, image_defocus.max()],
            rendering='average',
            depiction='plane',
            blending='additive',  # Changed to additive like projection planes
            opacity=0.8,
            plane={
                'position': config['position'],
                'normal': config['normal']
            }
        )
        print(f"\nAdded defocus microscope image to viewer at face {face}")
        print(f"  Plane position: {config['position']}")
        print(f"  Image 3D shape: {image_3d.shape}")
        print(f"  Intensity range: [{image_defocus.min():.2e}, {image_defocus.max():.2e}]")
    


Added defocus microscope image to viewer at face +x
  Plane position: (0, 64.0, 64.0)
  Image 3D shape: (1, 128, 128)
  Intensity range: [0.00e+00, 8.07e-02]

Added defocus microscope image to viewer at face +y
  Plane position: (64.0, 0, 64.0)
  Image 3D shape: (128, 1, 128)
  Intensity range: [0.00e+00, 8.15e-02]

Added defocus microscope image to viewer at face -z
  Plane position: (64.0, 64.0, 0)
  Image 3D shape: (128, 128, 1)
  Intensity range: [5.57e-04, 7.39e+01]


In [ ]:
faces = []
runner.microscopes = []

runner.to_numpy()

for i, face in enumerate(faces):
    runner.init_microscope(face, N, 1.0, 0.15, 1, 1.33, (N, N), 1.0, 550, 64.0)

    # Form image with defocus
    image_defocus = runner.defocus_image(i)

    # Debugging: print statistics
    print(f"\nMicroscope image statistics:")
    print(f"  Shape: {image_defocus.shape}")
    print(f"  Min: {image_defocus.min():.6e}")
    print(f"  Max: {image_defocus.max():.6e}")
    print(f"  Mean: {image_defocus.mean():.6e}")
    print(f"  Sum: {image_defocus.sum():.6e}")
    print(f"  Non-zero pixels: {(image_defocus > 0).sum()}")

    # Add to viewer as a plane at the correct face position
    if image_defocus.max() > 0:
        # Map observation face to plane position and normal
        # NOTE: Positions should be within [0, N-1] bounds
        face_config = {
            '+z': {'position': (N/2, N/2, 0), 'normal': (0, 0, -1)},   # Changed N to N-1
            '-z': {'position': (N/2, N/2, 0), 'normal': (0, 0, 1)},
            '+x': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},   # Changed N to N-1
            '-x': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},
            '+y': {'position': (N/2, 0, N/2), 'normal': (0, -1, 0)},   # Changed N to N-1
            '-y': {'position': (N/2, 0, N/2), 'normal': (0, 1, 0)},
        }

        config = face_config[face]
        
        # Reshape image to 3D for plane rendering (same as projection planes)
        if face in ['+x', '-x']:
            # z-face: image is in xy plane
            image_3d = image_defocus[np.newaxis, :, :]  # Shape: (1, Y, X)
        elif face in ['+z', '-z']:
            # x-face: image is in yz plane
            image_3d = image_defocus[:, :, np.newaxis]  # Shape: (Y, 1, Z)
        elif face in ['+y', '-y']:
            # y-face: image is in xz plane
            image_3d = image_defocus[:, np.newaxis, :]  # Shape: (X, Z, 1)
        
        viewer.add_image(
            image_3d,
            name=f'Microscope Defocus ({face})',
            colormap='gray',
            contrast_limits=[0, image_defocus.max()],
            rendering='average',
            depiction='plane',
            blending='additive',  # Changed to additive like projection planes
            opacity=0.8,
            plane={
                'position': config['position'],
                'normal': config['normal']
            }
        )
        print(f"\nAdded defocus microscope image to viewer at face {face}")
        print(f"  Plane position: {config['position']}")
        print(f"  Image 3D shape: {image_3d.shape}")
        print(f"  Intensity range: [{image_defocus.min():.2e}, {image_defocus.max():.2e}]")
    else:
        print("No signal in defocus microscope image")


In [ ]:
lines = runner.tracking_lines()
if len(lines) > 0:
    viewer.add_shapes(
        lines,
        shape_type='line',
        edge_width=0.1,
        edge_color='cyan',
        opacity=0.3,
        name='Photon Trajectories'
    )